In [18]:
import matplotlib.pyplot as plt
import pandas
import re
from pathlib import Path
import numpy as np
import random
import os

import Utilities

In [1]:
# Try this line if the cells below gives strange errors
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

^C


(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)


In [19]:
csv_categorized = Utilities.GetCSVDictionary()

CP1 - 120 
CP2 - 112 
CP3 - 111 
CP5 - 115 
CP6 - 103 
CP7 - 109 
CP8 - 102 
CP4 - 3 


In [6]:
from IPython.display import display, clear_output

def GoThroughValidityOfCP(CPNum):
    # create a dictionary recording if each path is valid
    # key: path   value: validity (bool)
    path_validity = {}
    CPNum = str(CPNum)
    zoomFactor = 50
    wantedDistanceWithin = 30

    plt.figure(figsize=(10,10))

    if CPNum not in csv_categorized:
        raise ValueError(f"No CP {CPNum}")

    for path in csv_categorized[CPNum]:
        # there are broken graphs so so try and except
        try:
            df = pandas.read_csv(path, sep=';')        
            dfA = df[((df['HeadPosXA'] > -wantedDistanceWithin) & (df['HeadPosXA'] < wantedDistanceWithin)) & ((df['HeadPosZA'] > -wantedDistanceWithin) & (df['HeadPosZA'] < wantedDistanceWithin))]
            dfB = df[((df['HeadPosXB'] > -wantedDistanceWithin) & (df['HeadPosXB'] < wantedDistanceWithin)) & ((df['HeadPosZB'] > -wantedDistanceWithin) & (df['HeadPosZB'] < wantedDistanceWithin))]
            plt.plot(dfA['HeadPosXA'], dfA['HeadPosZA'], color = 'red', alpha=0.5)
            plt.plot(dfB['HeadPosXB'], dfB['HeadPosZB'], color = 'blue', alpha=0.5)
            plt.plot([-wantedDistanceWithin,-wantedDistanceWithin],[-wantedDistanceWithin,wantedDistanceWithin], color='black')
            plt.plot([-wantedDistanceWithin,wantedDistanceWithin],[wantedDistanceWithin,wantedDistanceWithin], color='black')
            plt.plot([wantedDistanceWithin,wantedDistanceWithin],[wantedDistanceWithin,-wantedDistanceWithin], color='black')
            plt.plot([wantedDistanceWithin,-wantedDistanceWithin],[-wantedDistanceWithin,-wantedDistanceWithin], color='black')

            plt.xlim([-zoomFactor, zoomFactor])
            plt.ylim([-zoomFactor, zoomFactor])
            # display the current figure and ask valid or not
            display(plt.gcf())  
            validity = input("Is the graph valid? (y/n)")
            if(validity == "stop"):
                return path_validity  
            path_validity[path] = True if validity.lower() == 'y' else False
            
            # Clear the plot for the next graph
            plt.clf()
            clear_output()
            
        except Exception as e:
            #print(f"Failed to process {path} because of {e}")
            path_validity[path] = False
            continue
    return path_validity

In [15]:
my_path_validity = GoThroughValidityOfCP(8)

<Figure size 1000x1000 with 0 Axes>

In [16]:
df = pandas.DataFrame(list(my_path_validity.items()), columns=['Path', 'Validity'])

cwd = os.getcwd()

new_paths = []
for p in df['Path']:
    new_paths.append(str(Path(p).relative_to(cwd)))

df['Path'] = new_paths
folder_path = Path(cwd) / 'Data/DataValidity/path_validity.csv'

df.to_csv(str(folder_path), sep=';', index=False)